# Temperature plot Innsbruck

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import statsmodels.api as sm

### - Open the precipitation and temperature datasets

In [3]:
T_df = pd.read_table("/home/ciccuz/temperature_plot/HISTALP_AT_INN_T01_1760_2020.csv", sep=";", skiprows=13, index_col=0)
p_df = pd.read_table("/home/ciccuz/temperature_plot/HISTALP_AT_INN_R01_1760_2020.csv", sep=";", skiprows=13, index_col=0)

### - Some post-processing steps

In [4]:
#keep just the monthly values and not the seasonal ones
T_df1 = T_df.iloc[:,0:12]

#find all the data were no missing values are present
T_monthly = pd.DataFrame(index=range(1,2907), columns=['date', 'T_mean'])
T_mean = []
i=0
for year in T_df1.index:
    
    #array of values for the current year:
    x = T_df1.loc[year].loc[T_df1.loc[year].astype(np.float) != 999999]
    
    if x.size == 0:
        continue
    else:
        T_mean = pd.concat([pd.Series(T_mean), pd.Series(x.values)], axis=0).reset_index(drop=True)
        
        #cycle to pick the date: year and month
        for month in range(1,len(x)+1):
            T_monthly['date'][i+month] = pd.to_datetime(f'{year}{month}', format='%Y%m')

        i=i+12
        
T_mean.index += 1 

#multiply *0.1 to obtain actual degrees C
T_monthly['T_mean'] = T_mean*0.1

### - Choose the year range

In [17]:
#choose the 100 years on which calculate means, do the regression and plot: NOT LARGER THAN 1918!
first_year = 1918
index_first_year = T_monthly.loc[(T_monthly['date'].dt.year == first_year)].index[0]
index_last_year = T_monthly.loc[(T_monthly['date'].dt.year == first_year+100)].index[-1]

T_monthly_subset = T_monthly.loc[index_first_year : index_last_year].reset_index(drop=True)

### - Calculate the annual mean values, the linear regression and the deviation from 100-years mean

In [18]:
#Annual mean (mean values every 12 months)
T_annual_avg = pd.DataFrame(columns=['year', 'T_mean'])
T_annual_avg['year'] = pd.to_datetime(np.unique(T_monthly_subset['date'].dt.year), format='%Y')

for year,i in zip(np.unique(T_monthly_subset['date'].dt.year), range(0,101,1)):
    T_annual_avg['T_mean'][i] = np.mean(T_monthly_subset['T_mean'].loc[(T_monthly_subset['date'].dt.year == year)])


#Linear regression of monthly means values:
var_x = T_monthly_subset['date'].apply(lambda date: date.toordinal())
var_y = T_monthly_subset['T_mean']
data = np.transpose([var_x, var_y])
df_linreg = pd.DataFrame(data, columns=['date', 'T_mean'])

results = sm.OLS(df_linreg.T_mean, sm.add_constant(df_linreg.date)).fit()
par = results.params

#deviation of annual mean temperature from mean
dev_T = pd.DataFrame(columns=['year','T_dev'])
dev_T['year'] =  T_annual_avg['year']
dev_T['T_dev'] =  T_annual_avg['T_mean'] - np.mean(T_annual_avg['T_mean'])

# Plot with bokeh

In [19]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import SingleIntervalTicker, LinearAxis, tickers, Legend, ColumnDataSource,  DatetimeTickFormatter
from bokeh.layouts import gridplot

In [20]:
output_notebook()

Loading BokehJS ...

In [22]:
#INTERACTIVE PLOTS TOOLS
#linked planning
plot_options = dict(plot_width=950, tools='pan,wheel_zoom,box_select,lasso_select,help')

title = f'Temperature in Innsbruck {first_year}-{first_year+100}'
# create a new plot with a title and axis labels
p1 = figure(title=title, x_axis_type="datetime", plot_height=400,
            x_range=[T_annual_avg['year'][0], T_annual_avg['year'].iloc[-1]], **plot_options)
p1.title.text_font_size='20pt'

#plot the monthly means time-series
Tmm = p1.line(x=T_monthly_subset['date'], y=T_monthly_subset['T_mean'], 
              legend="T monthly avg ", line_width=2, alpha=0.5, muted_alpha=0.2)

#plot the yearly means time-series
Tym = p1.line(x=T_annual_avg['year'], y=T_annual_avg['T_mean'], legend="T annual avg ", 
              line_width=1.5, color = 'black', muted_alpha=0.2, muted_color='black')

#plot the average values over the 100 years
ymean = np.zeros(len(T_annual_avg['year']))+np.mean(T_annual_avg['T_mean'])
T100m = p1.line(x=T_annual_avg['year'], y=ymean, legend="T 100-years avg ", line_width=3, 
       color = 'green', alpha=0.5,  muted_alpha=0.2, muted_color='green')

#linear regression line:
ytrend = par.const + par.date * df_linreg.date
Ttrend = p1.line(x=T_monthly_subset['date'], y=ytrend, legend="T trend", line_width=2, 
                color = 'red',  muted_alpha=0.2, muted_color='red')

#set the x and y grid lines
p1.xgrid.grid_line_alpha=0.8
p1.ygrid.grid_line_alpha=0.8

#set y label
p1.yaxis.axis_label='°C'

#set the major and minor x ticks frequency
p1.xaxis[0].ticker.desired_num_ticks = 20
p1.axis.major_tick_out = 10

#customize the legend
p1.legend.orientation = "horizontal"
p1.legend.location = "top_center"
p1.legend.border_line_width = 1.5
p1.legend.border_line_color = "black"
p1.legend.border_line_alpha = 0.5

#interactive legend: by clicking on the legend element it is muted
p1.legend.click_policy="mute"


#add a second plot for the histogram of deviation of annual mean temperature from mean
p2 = figure(x_range=p1.x_range, x_axis_type="datetime", plot_height=200, **plot_options)

p2.vbar(x=T_annual_avg['year'], top=dev_T['T_dev'],  bottom=0, width=20000000000, fill_color='red',
                         color='black', legend='Annual mean temperature deviation from 100-years mean', 
                         muted_alpha=0.2, muted_color='red')

#customize the legend
p2.legend.orientation = "horizontal"
p2.legend.location = "top_center"
p2.legend.border_line_width = 1.5
p2.legend.border_line_color = "black"
p2.legend.border_line_alpha = 0.5

#set the major and minor x ticks frequency
p2.xaxis[0].ticker.desired_num_ticks = 20
p2.axis.major_tick_out = 10

fig = gridplot([[p1], [p2]])

show(fig)